In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.model_selection import cross_val_score,train_test_split,LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

from data_load import *
from dictionaries import *

We need to create our feature vector of exact same dimension as our training set. To convert our user input into dummy variables, we should save a dict of the the dummy variables. Later we can populate our feature vector for prediction using this dict.


# Test Vector for model testing

In [2]:
test_vector = [None] * 25
test_vector[0] =  2   # {'problem_type': {'defined': 0, 'investigate': 2, 'discover': 1}}
test_vector[1] =  2   #{'creative': {'somewhat_creative': 2, 'creative': 0, 'not_creative': 1}}
test_vector[2] =  1   #{'outdoors': {'limited': 1, 'indoors': 0, 'outdoors': 2}}
test_vector[3] =  4   #{'career': {'moving_parts': 2, 'sensors': 6, 'programming': 4, 'optimizing': 3, 'resources': 5, 'buildings': 0, 'molecules': 1}}
test_vector[4] =  1   #{'group_work': {'occasionally': 1, 'yes': 2, 'no': 0}}
test_vector[5] =  4   #{'liked_courses': {'computer_science': 4, 'biology': 1, 'math': 8, 'physics': 9, 'chemistry': 3, 'business': 2, 'autoshop': 0, 'geography': 5, 'visual_arts': 10, 'language_arts': 7, 'history': 6}}
test_vector[6] =  6   #{'disliked_courses': {'history': 6, 'visual_arts': 10, 'geography': 5, 'chemistry': 3, 'physics': 9, 'biology': 1, 'language_arts': 7, 'business': 2, 'math': 8, 'computer_science': 4, 'autoshop': 0}}
test_vector[7] =  3   #{'programming': {'partial': 3, 'complete': 0, 'no': 2, 'interested': 1}}
test_vector[8] =  8   #{'join_clubs': {'robotics': 7, 'nan': 6, 'environment': 4, 'student_council': 8, 'business': 2, 'hacker_club': 5, 'autoshop': 1, 'art/design': 0, 'consulting': 3}}
test_vector[9] =  0   #{'not_clubs': {'art/design': 0, 'business': 2, 'student_council': 8, 'autoshop': 1, 'hacker_club': 5, 'environment': 4, 'robotics': 7, 'nan': 6, 'consulting': 3}}
test_vector[10] = 1   #{'liked_projects': {'mars_water_treatment': 1, 'prototyping_instrument': 3, 'robot': 4, 'olympic_village': 2, 'battery': 0, 'uber_pool': 6, 'supercomputer': 5}}
test_vector[11] = 1   #{'disliked_projects': {'prototyping_instrument': 3, 'mars_water_treatment': 1, 'robot': 4, 'uber_pool': 6, 'olympic_village': 2, 'battery': 0, 'supercomputer': 5}}
test_vector[12] = 0   #{'tv_shows': {'big_bang_theory': 0, 'breaking_bad': 1, 'myth_busters': 4, 'silicon_valley': 6, 'planet_earth': 5, 'greys_anatomy': 2, 'house_hunters': 3}}
test_vector[13] = 0   #{'alternate_degree': {'cs': 2, 'applied_science': 0, 'health': 8, 'env': 5, 'poli_sci': 12, 'econ': 4, 'math': 11, 'business': 1, 'design': 3, 'visual_arts': 14, 'psych': 13, 'geo': 7, 'fin': 6, 'marketing': 10, 'lit': 9}}
test_vector[14] = 1   #{'expensive_equipment': {'yes': 2, 'maybe': 0, 'no': 1}}
test_vector[15] = 2   #{'drawing': {'partial': 2, 'bad': 0, 'good': 1}}
test_vector[16] = 1   #{'essay': {'yes': 2, 'no': 0, 'partial': 1}}
test_vector[17] = 0   #{'architecture': {0: 0, 1: 1}}
test_vector[18] = 0   #{'automotive': {1: 1, 0: 0}}
test_vector[19] = 1   #{'business': {0: 0, 1: 1}}
test_vector[20] = 0   #{'construction': {1: 1, 0: 0}}
test_vector[21] = 0   #{'health': {0: 0, 1: 1}}
test_vector[22] = 1   #{'environment': {0: 0, 1: 1}}
test_vector[23] = 1   #{'manufacturing': {1: 1, 0: 0}}
test_vector[24] = 1   #{'technology': {1: 1, 0: 0}}

test_vector = np.array(test_vector).reshape(1, -1)

# Basic Naiyve Bayes Model with no Calibration

In [3]:
model_name = 'nb_model_basic'

In [4]:
data = get_encoded_data('quiz_data.csv','nb_model_basic',True)[0]

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
Y = np.array(y_df) # convert dataframe into np array

mnb = MultinomialNB()
model = mnb.fit(X, Y) # fit the model using training data

cat = data.drop('program',axis=1)
cat = dict(zip(cat.columns,range(cat.shape[1])))

print(INV_INDEX_PROGRAM[model.predict(test_vector)[0]])

save_model(model,cat,model_name)
test_model(model_name,test_vector)

nano
Loading CAT file...
Loading model...
Results:
{'arch': 0.0, 'arch-e': 0.0, 'bmed': 0.0433, 'ce': 0.0203, 'chem': 0.1727, 'cive': 0.0003, 'elec': 0.0425, 'env': 0.0011, 'geo': 0.0006, 'mech': 0.0261, 'msci': 0.031, 'nano': 0.5796, 'swe': 0.0049, 'syde': 0.008, 'tron': 0.0695}


### Model Accuracy for Basic Naive Bayes Model with no Calibration

In [5]:
# Leave One Out Validation
loo = LeaveOneOut()
loo.get_n_splits(X)
accuracy = []
for train_index, test_index in loo.split(X):
  X_train, X_test = pd.DataFrame(X[train_index]), pd.DataFrame(X[test_index]) # use this for training the model
  y_train, y_test = Y[train_index].ravel(), Y[test_index].ravel() # use this for testing the model

  mnb = MultinomialNB()
  model = mnb.fit(X_train, y_train) # fit the model using training data
  accuracy.append(mnb.score(X_test, y_test))

# Calculate accuracy
mean = np.array(accuracy).mean()
variance = np.array(accuracy).std() * 2

print("nb_model_basic Accuracy: %0.4f (+/- %0.4f)" % (mean, variance))

nb_model_basic Accuracy: 0.3734 (+/- 0.9674)


# Test

In [6]:
get_encoded_dict(model_name)

{'program': {'program': {'mech': 9,
   'bmed': 2,
   'swe': 12,
   'tron': 14,
   'cive': 5,
   'chem': 4,
   'syde': 13,
   'msci': 10,
   'ce': 3,
   'elec': 6,
   'nano': 11,
   'geo': 8,
   'env': 7,
   'arch-e': 1,
   'arch': 0}},
 'problem_type': {'problem_type': {'defined': 0,
   'investigate': 2,
   'discover': 1}},
 'creative': {'creative': {'somewhat_creative': 2,
   'creative': 0,
   'not_creative': 1}},
 'outdoors': {'outdoors': {'limited': 1, 'indoors': 0, 'outdoors': 2}},
 'career': {'career': {'moving_parts': 2,
   'sensors': 6,
   'programming': 4,
   'optimizing': 3,
   'resources': 5,
   'buildings': 0,
   'molecules': 1}},
 'group_work': {'group_work': {'occasionally': 1, 'yes': 2, 'no': 0}},
 'liked_courses': {'liked_courses': {'computer_science': 4,
   'biology': 1,
   'math': 8,
   'physics': 9,
   'chemistry': 3,
   'business': 2,
   'autoshop': 0,
   'geography': 5,
   'visual_arts': 10,
   'language_arts': 7,
   'history': 6}},
 'disliked_courses': {'disliked_c

In [7]:
encoded_dictionary = get_encoded_dict(model_name)

program = encoded_dictionary['program']
problem_type = encoded_dictionary['problem_type']
creative = encoded_dictionary['creative']
outdoors = encoded_dictionary['outdoors']
career = encoded_dictionary['career']
group_work = encoded_dictionary['group_work']
liked_courses = encoded_dictionary['liked_courses']
disliked_courses = encoded_dictionary['disliked_courses']
programming = encoded_dictionary['programming']
join_clubs = encoded_dictionary['join_clubs']
not_clubs = encoded_dictionary['not_clubs']
liked_projects = encoded_dictionary['liked_projects']
disliked_projects = encoded_dictionary['disliked_projects']
tv_shows = encoded_dictionary['tv_shows']
alternate_degree = encoded_dictionary['alternate_degree']
expensive_equipment = encoded_dictionary['expensive_equipment']
drawing = encoded_dictionary['drawing']
essay = encoded_dictionary['essay']
architecture = encoded_dictionary['architecture']
automotive = encoded_dictionary['automotive']
business = encoded_dictionary['business']
construction = encoded_dictionary['construction']
health = encoded_dictionary['health']
environment = encoded_dictionary['environment']
manufacturing = encoded_dictionary['manufacturing']
technology = encoded_dictionary['technology']

In [35]:
print(len(row))

2
